# DSCI 521: Data Analysis and Interpretation

# Term Project

## Team Members

- Team member 1
    - Name: Andrew Appleton
    - Email: aa4498@drexel.edu    
    - Background and skills: 10 years experience in Health Sciences, Limited Python experience
- Team member 2
    - Name:  Layla Bouzoubaa
    - Email: lb3338@drexel.edu
    - Background and skills: MS in Public Health with a concentration in Biostatistics. Advanced R user, novel Pythonista, strong familiarity with public health data science pipeline and research.
- Team member 3
    - Name:  Alicia Brandemarte
    - Email: amb847@drexel.edu
    - Background and skills: Healthcare Informatics, R, SQL, Business Intelligence.  Limited Python skills, seeking to expand Python Skills.
- Team member 4
    - Name: Stephan Dupoux
    - Email: sgd45@drexel.edu    
    - Background and skills: 5 years of building machine learning , 5 years of Python programming such as building ETL pipelines and other web based applications. 
- Team member 5
    - Name: Mary Lucas
    - Email: mml367@drexel.edu
    - Background and skills: Extensive healthcare clinical experience, some experience in SQL, Python, and R programming and machine learning. 
- Team member 6
    - Name: Shane Nelson
    - Email: sn888@drexel.edu
    - Background and skills: 2 years experience working at a clinical research organization. Somewhat limited Python experience however continuously seeking to improve skills.


# 1. Introduction

The COVID-19 pandemic is an ongoing global crisis that has had a major impact on individuals and communities over the past two years.  It has resulted in significant disruptions to our way of life and the services and resources we depend on on a day to day basis.  

The pandemic and its effects in terms of loss to life and livelihood have also made it more evident that there are disparities in access to care and resources across the globe based on geography, race, and other factors.
  
All code can be found on our [Github repository](https://github.com/labouz/covid_vulnerability/)  The data is available for download from the links in the Data Sources section.

### Motivating Questions to Address
The main questions that we hoped to address in this work were: 

  * the impact COVID-19 has had on hospital capacity in different regions of the country over time
  * the community burden of the COVID-19 pandemic
  * health disparities in the COVID-19 impact across different regions of the country



### Design of the Study

This study is a descriptive and analytic design of time series data to understand and predict hospitalization burden due to the COVID19 pandemic in different states.  We implemented time series forecasting methods and modelling of hospital burden across the country due to the COVID-19 pandemic. 

Data used was accessed from open data repositories of the federal government at healthdata.gov and data.gov, and census.gov, as well as supplemental sources such as the Kaiser Family Foundation. Analysis was conducted using standard time series focusing and regression and classification methods.

No costs were incurred and no human subjects were involved in the study.

### Report Breakdown 

To make the individual files more manageable we have divided our across two notebooks. This current notebook contains the written report and embedded images of visualizations generated from our analysis and modelling.  

The code that is needed to reproduce our work is contained in a separate notebook project_code.ipynb.  

# 2. Data


The main data we used was obtained from The US Department of Health and Human Services (HHS) as healthdata.gov and is publically available. The datasets are all non-identifiable and contain no protected health information. 

### Data Sources

The main dataset used for this analysis is the hospital capacity timeseries data from the healthdata.gov website. This data is updated daily and can be downloaded from [COVID-19 Reported Patient Impact and Hospital Capacity by State Timeseries](https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh)

The second dataset is a community vulnerability ranking dataset, by county, provided at data.gov. This data is fixed in time and has not been updated since at least November 2021 according to the metadata on the data page. It can be downloaded from [COVID-19 Community Vulnerability Crosswalk - Rank Ordered by Score](https://catalog.data.gov/dataset/covid-19-community-vulnerability-crosswalk-rank-ordered-by-score)

Supplementary data on population metrics, socioeconomic status, and hospital counts in different states was obtained from other sources and will be referenced in the relevant sections of this report.


### Data Structure
The hospital impact and capacity timeseries data and the community vulnerability raking data is provided in various formats including CSV, TSV, GEOJSON, XML, and more.  It is updated on a daily basis and can be downloaded via the Socrata API or by direct export from the links provided.  As of the last download of the final dataset for this project the file contained 39,701 rows and 117 columns with each row representing a daily state aggregated report. Because of API limitations we opted to download a CSV file of the data for our work.

The other datasets were all provided either as Excel/CSV files or were in table format on a webpage or a PDF and needed to be scraped and saved in CSV format.  

All datasets are all included in the data folder that accompanies this report.

# 3. Exploratory Data Analysis


In [10]:
# code to allow for visualizations
import warnings
import IPython
from IPython.display import Image
warnings.filterwarnings('ignore')

.... **ALICIA**

...


...

In [5]:
url = 'https://marymlucas.github.io/adult-icu_utilization/'
iframe = '<iframe src=' + url + ' width=100% height=600></iframe>'
IPython.display.HTML(iframe)

# 3. Methods

## 3.1 Data Preprocessing
We undertook various steps to prepare our data. These are outlined below.

### Data Cleaning
This was an iterative process that involved exploring the data, noting anomalies and removing them, and then exploring the data again, and therefore these steps were not completed in a sequential manner.

The main cleaning measures were:

- date confinements:  we noticed that the data was very sparse before 3/1/2020 and therefore decided to set that as the start date of our analysis. We settled on an even 24 months for analysis and therefore constrained our dataset to this time period.

- anomalous data points:  we removed one data point for the state of Iowa where the adult ICU bed utilization (our main metric of interest) was reported as greater than 50.  Because utilization is a proportion measure, we expected to see utilization values between 0 and 1.  The majority of observations fell in this range as evidenced by the scatter plot in fig x. There were a few that were above 1 and even fewer above between 2 and 3.  We considered that given the nature of the pandemic and the extent to which hospitals were sometimes overrun, values of utilization above 1 would indicate that some ICUs were running over capacity and overflowing into other areas of the hospital, hence utilization > 1. Because we only saw a value higher than 3 only once in this data (Iows on 08/15/2020), we decided it was likely a clerical error and opted to remove this row from our data. 

### Feature Selection
After an initial data exploration phase, we selected 35 variables out of the 117 to retain for our analysis. This selection was made based on examination of variable collinearity and domain knowledge. We noticed that many of the highly correlated variables were also derived from each other. For example there were features defined as "numerator" and "denominator" which were not relevant to our analysis. Our goal was to analyze impact of hospital capacity and state-wide burden, therefore many features examining covid diagnosis (suspected and confirmed) were dropped because they were not critical to our analysis. In addition, many features were focused on pediatric measures that were not applicable to our analysis, considering our ultimate target variable for icu bed utilization. 

### Defining and Calculating Under-Reporting
To make our analysis meaningful, we decided to remove states that had high levels of underreporting.  For each metric in each row (day of data reported in the state), there is a corresponding "coverage" variable (denoted as *metric_coverage*). This coverage feature reports how many hospitals in the state submitted information about that metric on the day. For example, in the case of our main feaure of interest (adult_icu_bed_utilization), the *adult_icu_bed_utilization_coverage* feature gives an indication of how many hospitals in the state reported their adult icu bed utilization that day. 

By comparing the coverage with the number of hospitals in each state we were able to derive a simple measure of the level of reporting for each state.

The most current list of hospital numbers was obtained from the Kaiser Family Foundation website (https://www.kff.org/other/state-indicator/total-hospitals) which reports data from the American Hospital Association (1999 - 2019) Annual Survey. 

It's important to note the disclaimer that "Data are for community hospitals, which represent 85% of all hospitals" where community hospitals are defined as "all nonfederal, short-term general, and specialty hospitals whose facilities and services are available to the public."  Because of this, we expected that for states with high reporting we may have instances of greater than 100% reporting, which we decided was acceptable for our analysis.

In selecting a threshold, we considered the guidelines by the HHS for hospitals on how to report their numbers - https://www.hhs.gov/sites/default/files/covid-19-faqs-hospitals-hospital-laboratory-acute-care-facility-data-reporting.pdf. While this is a recent update, we noted that the changes from previous requirements did not affect the requirement to report or the frequency of reporting, only what fields are reported (https://healthdata.gov/stories/s/COVID-19-Reporting-and-FAQS/kjst-g9cm/). Based on this and on exploration of the data, we came up with a definition of under-reporting as "greater than 25% of the hospitals not reporting acute_icu_bed_utilization."  After running our code, we the states of Rhode Island and New York, as they had the largest number of days of underreporting over the two year perion in our analysis.  In addition, after plotting a boxplot of total days reported per state, we noted that the US territories of Virgin Islands and American Samoa were outliers and had very low reporting days, so we dropped the territories from our analysis. 

### Defining and Calculating Burden
We then calculated a burden metric for each state on each day.  To calculate burden, we used the criteria set by The US Department of Health and Human Services.  According to French et.al., (2021), “HHS has studied the relationship between hospital bed use and hospital strain and has identified occupancy >80% as an indicator of a strained condition. This analysis uses a continuous measure of ICU bed occupancy as a proxy for hospital strain, such that greater amounts of ICU bed use indicate larger amounts of hospital strain.” 

We determined from our survey of the relevant literature that 80% threshold is a good, validated metric for hospital strain that we could use as an indicator for our analysis.

The burden metric is binary 1/0 value that is 1 if the adult icu bed utilization is greater than 0.8, and less otherwise. 

## 3.2 Analysis and Modelling


### Time Series Analysis - Simple Moving Average
There are several factors that should be considered when performing a time series analysis. 

- Stationarity: a time series model is said to be stationary if its statistical properties do not change over time. In other words, it has constant mean and variance, and covariance is independent of time. 
- Auto-correlation: this is the similarity between observations as a function of time given a lag of time between them. 
- Seasonality: a period fluctuation of observations over time. 

Ideally, it is desirable to have a stationary time series for model. We can test for stationarity by performing a Dickey-Fuller test where the null hypothesis is that the series is not stationary (p > 0), and if p = 0 we then reject null hypothesis.

Our first model, the moving average, was the most naive. This model is simple in that it states that the next observation is the mean of all past observations in a given window. This model is typically a good starting point as we can identify interesting trends. In our analysis we hypothesized that our data is not stationary given that we observed the significant jump in ICU bed utilization around July 2020 (see animated choropleth).

### Regression and Classification for Predicting Burden
.... **ALICIA**

### Time Series Forecasting for Predicting Trends in Utilization
The goal here was to use standard forecasting methods to predict trends in ICU bed utilization during the pandemic. We chose to focus on Pennsylvania as this is our state of residence and where Drexel University is located. 

The data for PA was extracted and split into train and test subsets based on date, with the first 18 months (March 1, 2020 -  Sep 1, 2021) used as the training set and the last 6 months (Sep 1, 2021 - March 1, 2022) as the test set.  We then ran 3 different forecasting models on the data.  For simplicity, no adjustments, e.g. for stationarity, were made on either the training or the testing data.

The three models tested are described below:
- Autoregressive Moving Average (ARMA) 
    - doesn’t capture seasonal trends. 
    - assumes that the time series data is stationary, meaning that its statistical properties wouldn’t change over time. 
- Autoregressive Integrated Moving Average (ARIMA)
    - doesn’t assume stationarity 
    - assumes the data exhibits little to no seasonality
- Seasonal ARIMA (SARIMA)
    - is a variant of ARIMA 
    - can work with non-stationary data 
    - can capture some seasonality

The performance of each model was evaluated by calculating the root mean square error, and we also visualized the predicted trends compared to the actual trends.


### Disparity Analysis

For the disparity analysis, we were limited in what measures of disparity we could meaningfully use given the varying granularity of our different datasets. For example, most income and race data is reported at a county level (for example by the Census bureau). But our utilization timeseries data is on a state level. Because the counties in each state have different socioeconomic levels, it's difficult to aggregate these measures and analyse disparity at a state level in a meaningful and actionable way. 

We mapped each several indicators of community wellness in choropleths as a way to assess disparity and to compare the differences between states with the animated choropleth of ICU burden.  The mapped indicators are:

- Social Deprivation Index (SDI):
    - bla bla bla
- COVID vulnerability Level: measure 
    - (based on data that was last updated 11/2021)
multidimensional deprivation index.  We also mapped the population density of each state as an additional measure.



## 4. Results


### Time Series - Simple Moving Average

We tested the hypothesis that our time series was stationary by applying the Dicky-Fuller test and got a p-value = 0. Additionally, we observed very high autocorrelation suggesting no clear seasonality.

<table>
    <tr>
        <td> <img src="images/DF.png" alt="Drawing" style="width: 500px;"/> 
            <br>
        </td>
        <td> <img src="images/DF-auto.png" alt="Drawing" style="width: 500px;"/> 
            <br>
        </td>
    </tr>
</table>

Although we fail to reject the null hypothesis, when plotting the moving average line, with a 7 day window, against the data with confidence intervals, we can observe a downward trend at the end. If we were to stop here we could say that the utilization rate is likely to go down in the following days. 

<img src="images/simple_moving_average_7_day.png" alt="Drawing"/>

However, given this approach is not a very comprehensive method of anticipating burden, we decided to move on from the moving average model to try other nodels that could tell us more. If we were to stay on the path of the moving average model, we could have utilized other smoothing methods, like exponential or double exponential, to try to observe more interesting trends followed by adjusting the data to account for possible seasonality and stationarity by taking the first difference.


### Regression and Classification Modeling

... **ALICIA**

### Time Series Forecasting - ARMA, ARIMA, SARIMA

*bla bla bla bla bla text here*

<table>
    <tr>
        <td> <img src="images/ARIMA_PA.png" alt="Drawing" style="width: 500px;"/> 
            <br>
            <center>Comparing ARMA and ARIMA predictions with actual trend</center>
        </td>
        <td> <img src="images/SARIMA_PA.png" alt="Drawing" style="width: 500px;"/> 
            <br>
            <center>SARIMA prediction</center>
        </td>
    </tr>
</table>



### Disparity Analysis

We were able to include the social deprivation index for each state in our data and this variable was used in the regression and classification modelling that has been described and proved to have a predictive effect on the hospital burden.  

*bla bla bla bla bla more text here*



<table>
    <tr>
        <td> <img src="images/mdi.png" alt="Drawing" style="width: 500px;"/> 
            <br>
            <center>Multidimensional Deprivation Index (MDI)</center>
        </td>
        <td> <img src="images/covid_vulnerability.png" alt="Drawing" style="width: 500px;"/> 
            <br>
            <center>COVID Vulnerability Level (Hardest Hit Areas HHA)</center>
        </td>
    </tr>
</table>


*bla bla bla bla bla more text here*


<table>
    <tr>
        <td> <img src="images/sdi.png" alt="Drawing" style="width: 500px;"/> 
            <br>
            <center>Social Deprivation Index (SDI)</center>
        </td>
        <td> <img src="images/popn_density.png" alt="Drawing" style="width: 500px;"/> 
            <br>
            <center>Population Density</center>   
        </td>
    </tr>
</table>


*bla bla bla bla bla more text here*


## 5. Strengths and Limitations

### Strengths:

The strengths of this project lie in the amount of freely available data that we were able to harness and include in our analysis and modeling.  This shows that this kind of study can be extended and expanded on beyond our current work, and as the pandemic continues it can be easily updated and enhanced.  

Additionally, this analysis is timely and of great relevance to various stakeholders both in looking back and looking forwards. Understanding how different communities were impacted and burdened by the COVID-19 pandemic and some of the underlying factors that caused the disparate impact, such as socioeconomic factors (as measured by the social deprivation index) and population density can help policy makers and communities to prepare more robustly for future health emergencies or other disasters.


### Limitations:

We do have some limitations in our analysis and modelling that we would like to acknowledge. 

The primary limitation we faced, which has been alluded to in the disparity analysis section, is that we are using datasets that are provided at different granularity levels.  For example, the utilization data in our hospital time series data was at the state level while most of the disparity and social indicator data we could obtain was provided at the county level (apart from the SDI).  In future, it would be interesting to analyse timeseries information at the County level and then use Census data to, for example,  map socioeconomic indicatirs for the different counties. This would allow a more useful calculation and analysis of health disparities between groups.

A second limitation is that we had to use data points from different data collection timeframes. While our ICU utilization data is very real time, the disparity and socioeconomic data is not, it's frozen at the collection time (2017 for the MDI data and 2018 for the SDI data). Additionally, the population density data is based on the 2020 census.  We are therefore making the assumption that these metrics have not varied much over the past few years.  We did find that even though the SDI data we used was from 2017 it was still very predictive of hospital burden.

## 6. Conclusion

Our goal throughout this project was to come up with a model to understand the hospital burden throughout the U.S. during the COVID-19 pandemic and how socioeconomic disparities, and other metrics like population density may have impacted and continu to contribute to this burden.  We implemented multiple regression and time series models and were able to validate that social deprivation index and population density seemed to have a significant impact on how much hospital burden was experience din different states across the burden.

We concluded that the best performing model for predicting hospital burden was the autoregressive moving average (ARMA) model as it was able to predict correctly how ICU utilization would trend for the next six months. It also had the lowest root mean square error.  We hypothesize that we could use the ARMA model on a live dashboard that would update based on daily utilization data, keeping policy majers and public health administrators and other stakeholders updated and able to prepare for burden in their communities and to implement appropriate disaster management measures and clinical plans if necessary.


## References

<div class="csl-entry">French, G., Hulse, M., Nguyen, D., Sobotka, K., Webster, K., Corman, J., Aboagye-Nyame, B., Dion, M., Johnson, M., Zalinger, B., &#38; Ewing, M. (2021). Impact of Hospital Strain on Excess Deaths During the COVID-19 Pandemic — United States, July 2020–July 2021. <i>MMWR. Morbidity and Mortality Weekly Report</i>, <i>70</i>(46), 1613–1616. https://doi.org/10.15585/MMWR.MM7046A5</div>